<!--img src="images\cloudflow.png" width=1400 /-->

# Introduction 


## How to use this notebook
This example runs Sima in locally based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. 

This note bookshould be run in the following way:
1. [Installation](#installation) can be run to install all the relevant tools, like python modules and local worker executable
2. [Initialize Workflow](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up custom user parameters](#custom) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Set up OneWorkflow client](#builder).
3. [Run analysis](#run) shall be run each time a new Sima analysis needs to be run.
4. [Run Wasim and Sestra](#runwasim) run Wasim and Sestra using results from above Sima run.

### Postprocessing
It is now possible to read the SIN/FEM file using SifIO, please consult the C# [documentation](https://sesam.dnv.com/dev/api/sifio/). This notebook provides a small example script for getting node coordinates and displacements for the 200 first nodes for loadcase 11, [postprocessing.py](Workspace/CommonFiles/postprocessing.py). The script will be run just after Wasim and Sestra for each load case, see the [Run Wasim and Sestra](#runwasim) section. The section [Post processing](#postprocessing) shows how the post processed result files may be read and visualized in this notebook. More Python examples will be provided later. Relevant additional documents are:
- The input interface file documentations can be found [here](https://sesam.dnv.com/download/windows64/sesam_input_interface_format.pdf)
- The results interface format [here](https://sesam.dnv.com/download/windows64/sesam_results_interface_format.pdf)

The Python package should be installed with this command : pip install -i https://test.pypi.org/simple/ dnv-sifio --user as done in the [Installation](#installation) section.

### Hints
If you get this message "#ERROR - Error in WorkerHost: Completed, but failed to download one or more files:", please remember the upload common file step.

# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Installation  <a id='installation'></a>
Run only when packages and/or local worker  needs to be upgraded. Set _update_packages_ to True first time and when packages should be updated.

In [16]:
update_packages = True
import os
print(os.getcwd())
if update_packages:
    %pip install msa azure-storage-blob cancel-token asyncio httpx dacite stringcase pyjwt overrides python-dateutil pythonnet
    %pip install --index-url https://test.pypi.org/simple/ dnv-onecompute==0.1.0.527 dnv-oneworkflow==0.1.0.527 dnv-sesam-commands==0.1.0.527 --force-reinstall
    %pip install -i https://test.pypi.org/simple/ dnv-sifio --user
    # Python.NET for the seamless integration with the .NET Common Language Runtime (CLR)
    %pip install pythonnet --user
    # QuantConnect for intellisense for the .NET libraries
    %pip install quantconnect-stubs --user
    # SifIO for reading and writing SIF
    %pip install -i https://test.pypi.org/simple/dnv-sifio==0.0.3 --user

    #! pip install --upgrade --index-url https://test.pypi.org/simple/ dnv-onecompute=0.1.0.525
    #! pip install --upgrade --index-url https://test.pypi.org/simple/ dnv-oneworkflow=0.1.0.525
    #! pip install - -upgrade - -index-url https: // test.pypi.org/simple / dnv-sesam-commands = 0.1.0.525
    from oneWorkflowToolBox import *
    await install_workflow_runtime()


C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace



Looking in indexes: https://test.pypi.org/simple/Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement asyncio (from dnv-onecompute) (from versions: none)
ERROR: No matching distribution found for asyncio



  Using cached https://test-files.pythonhosted.org/packages/58/c7/e34f974f924472c6f203deedb644048068fe1fdbf3f077a91af513ad566d/dnv_onecompute-0.1.0.527-py3-none-any.whl (32 kB)
  Using cached https://test-files.pythonhosted.org/packages/aa/2c/f028b357afa90b10f624cb8d2dfce5edda99a72e41b19725a8c8f1a2ebe7/dnv_oneworkflow-0.1.0.527-py3-none-any.whl (24 kB)
  Using cached https://test-files.pythonhosted.org/packages/d9/56/f5024b6f08175473186bf89d358762d282a7f76f729b5bf82a6002614797/dnv_sesam_commands-0.1.0.527-py3-none-any.whl (6.5 kB)
  Using cached https://test-files.pythonhosted.org/packages/a4/9e/7b2278305feab76061245761000046b9493ce687e97c82550c4e2e6a30d1/msal-1.22.0-py2.py3-none-any.whl (90 kB)
  Using cached https://test-files.pythonhosted.org/packages/3e/84/610f379b46d7d3c2d48eadeed6a12b6d46a43100fea70534f5992d0ac996/azure_storage_blob-2.1.0-py2.py3-none-any.whl (88 kB)
  Using cached https://test-files.pythonhosted.org/packages/89/40/6e70d6e17dccbcb36b430a3af26c7b301e902f4b8784bdb

Note: you may need to restart the kernel to use updated packages.


ERROR: You must give at least one requirement to install (see "pip help install")
ERROR: Failed to install as the directory 'C:\Users\kblu\AppData\Local\OneCompute\LocalWorkflowRuntime' is locked by another process


Note: you may need to restart the kernel to use updated packages.


## Set up custom user parameters <a id='custom'></a>

In [17]:
import os
# local workspace, all results will be put here after local or cloud runs
workspacePath = r'C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace'
# location of common files for all analysis, has to be below workspacePath
workspaceId = "SE28"
loadcase_file = f"{workspacePath}\\test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"
cloud_run = False
notebook_root_folder = os.getcwd()

## Set up OneWorkflow client <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [20]:
from oneWorkflowToolBox import *
workflow_client = one_workflow_client(
    workspaceId, workspacePath, cloud_run, tmp=r"c:\OneWorkflowTemp") 
workspace = workflow_client.one_workflow_config.workspace_config
commonfiles_folder = workspace.common_files_directory
results_folder = workspace.results_directory
#If running locally the code below will also start the local workflow host.
if (cloud_run):
    workflow_client.login()


The temporary blob storage directory is: c:\OneWorkflowTemp\oc_yfvyoo9r_blob
The temporary jobs root directory is: c:\OneWorkflowTemp\oc_jq8awkh7_jobs
c:\OneWorkflowTemp


## Upload common files for the job <a id='upload'></a>
This step uploads all common files in folder *commonFilesDirectory*  to the job. Only needed to run if new common files are to be uploaded or workspace changed.

In [23]:

from dnv.onecompute.directory_client import FileOptions
try:
    workflow_client.upload_common_files(FileOptions(
        # max_size_bytes=124_000,
        #patterns=["**/*.py","**/*.inp"],
        overwrite=True))
except Exception as e:
    print(e)
    print("Ignore this error message if the files are already present.")


Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\copyFiles.py to c:\OneWorkflowTemp\oc_yfvyoo9r_blob\SE28\CommonFiles\copyFiles.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\Floater.ssg to c:\OneWorkflowTemp\oc_yfvyoo9r_blob\SE28\CommonFiles\Floater.ssg
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\postprocessing.py to c:\OneWorkflowTemp\oc_yfvyoo9r_blob\SE28\CommonFiles\postprocessing.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\requirements.txt to c:\OneWorkflowTemp\oc_yfvyoo9r_blob\SE28\CommonFiles\requirements.txt
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\sestra_template.inp to c:\OneWorkflowTemp\oc_yfvyoo9r_blob\SE28\CommonFiles\sestra_template.inp
Uploading C:\Us

# Run analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [24]:
from oneWorkflowToolBox import run_workflow_async
from SimaHelper import *
import json
"""Tests SIMA and Python commands"""
# Upload Input Files
workflow_client.upload_input_files()
#Sima path must be specified
sima_settings = SimaSettings(
    sima_exe_path=r'C:\Program Files\DNV\Sima V4.4-00')
sima_work_unit = SimaTaskCreator(
    sima_settings, workflow_client).get_sima_work_unit(loadcase_file, stask_file)
# Create Parallel Work Unit and Job
job = workflow_client.create_job(sima_work_unit)

job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
# Run workflow
downloadOptions = FileOptions(
    max_size_bytes=11124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg"])
await run_workflow_async(job, workflow_client, downloadOptions)


Error: load cases directory C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\LoadCases does not exist.
The work item c91b7318-95d4-4467-a8ea-2745d2297ce7 message is 'Created'
The work item test001 message is 'Created'
The work item test002 message is 'Created'
The work item test003 message is 'Created'
The progress of the job is '0%'. The message is ''
The work item test001 message is ''
The work item test002 message is ''
The work item test003 message is ''
The work item test001 message is '2023-06-05 11:54:54,932 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMALog.java:51)'
The work item test002 message is '2023-06-05 11:54:54,927 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMALog.java:51)'
The work item test003 message is '2023-06-05 11:54:55,327 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMA

# Run Wasim and Sestra <a id='runwasim'></a>

In [25]:
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
from dnv.sesam.wasim_command import WasimStruCommand,  WasimSetupCommand
from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import Platform
import json
import os
from dnv.sesam.sestra_command import SestraCommand
workspace.results_directory = "Results"
load_cases = ["test001", "test002"]

os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {
    'FMT': topSuperElement, 'topsel': topSuperElement}

def run_wasim_and_sestra_using_results_from_sima(
        results_directory: str, load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    parallel_work_units = list[WorkUnit]()

    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue
        
        load_case_result_files_dir = os.path.join(results_directory, casename)
        #For testing
        #python_copy_command = PythonCommand(
        #    directory=commonfiles_folder,
        #    filename="copyfiles.py",
        #    working_dir=load_case_result_files_dir)
        casedict = case.to_dict()
        
        cmd = WasimAndSestraTaskCreator(
            load_case_result_files_dir, commonfiles_folder, casedict, additionalTemplateParameters).CreateTasks()
        work_unit = (
            WorkUnit(cmd, f"post_rerun_{casename}")
            .input_directory(load_case_result_files_dir)
            .output_directory(load_case_result_files_dir, include_files=["**/sima.*", "**/*.txt", "**/*.tda", "**/*.bin", "**/*.log", "**/*.inp", "**/*.lis", "**/*.mlg", "**/*.sin"])
        )
        parallel_work_units.append(work_unit)

    return ParallelWork(parallel_work_units)
print(os.getcwd())
print(workflow_client.workspace_path)

work_unit = run_wasim_and_sestra_using_results_from_sima(
    workspace.results_directory, load_cases)
if not cloud_run:
    workflow_client.upload_common_files()
    workflow_client.upload_result_files()
print(workflow_client.temp_path)


downloadOptions = FileOptions(
    min_size_bytes =0,
    max_size_bytes=11124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg", "**/*.sin"])
#await run_workflow_async(workflow_client.create_job(work_unit), workflow_client, downloadOptions)
await workflow_client.run_workflow_async(
    work_unit,
    work_item_status_changed=work_item_status_changed_callback(workflow_client, downloadOptions),
    log_job=True
)


C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace
C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace
skipping test003
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\copyFiles.py to c:\OneWorkflowTemp\oc_yfvyoo9r_blob\SE28\CommonFiles\copyFiles.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\Floater.ssg to c:\OneWorkflowTemp\oc_yfvyoo9r_blob\SE28\CommonFiles\Floater.ssg
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\postprocessing.py to c:\OneWorkflowTemp\oc_yfvyoo9r_blob\SE28\CommonFiles\postprocessing.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\requirements.txt to c:\OneWorkflowTemp\oc_yfvyoo9r_blob\SE28\CommonFiles\requirements.txt
Uploading C:\Users\kblu\sourc

'37c90f7c-5312-46be-ae1b-08b49e3dddf0'

# Post processing  <a id='postprocessing'></a>

In [27]:
# imports
from IPython.display import display
import numpy as np
import pandas as pd
import glob
from ipywidgets import interactive

lc = 11
dataFrames = {}
for folder in glob.glob(f"{notebook_root_folder}\\Workspace\Results\\*"):
    test_name = folder.split("\\")[-1]
    try:
        data = np.loadtxt(f"{folder}\\postprocessedresultsLC{lc}.txt")
        dispdata = {
            "x-coord": data[:, 0],
            "total-disp":  np.sqrt(data[:, 3]**2+data[:, 4]**2+data[:, 5]**2)
        }
        dataFrames[test_name] = pd.DataFrame(dispdata)
    except:
        print("Failed loading data for test :" + test_name)

def multiplot(resultcase):
    dataFrames[resultcase].plot(
        x="x-coord", y="total-disp", kind="scatter", figsize=(15, 15))


interactive_plot = interactive(
    multiplot, resultcase=dataFrames.keys())
interactive_plot


Failed loading data for test :test003


interactive(children=(Dropdown(description='resultcase', options=('test001', 'test002'), value='test001'), Out…